# 02: Feature Engineering with Foundation Models & Vector Search

This notebook modernizes our feature engineering pipeline by using Databricks Foundation Model APIs for embeddings and setting up Unity Catalog Vector Search for real-time inference.

**Upgrades:**
- **Embedding Model**: Uses `databricks-gte-large-en` for superior semantic representation.
- **Vector Database**: Configures and populates a Unity Catalog Vector Search index for fast, scalable similarity searches, which is essential for our agent's tools.


In [0]:
%pip install databricks-vectorsearch databricks-sdk --upgrade
dbutils.library.restartPython()


In [0]:
dbutils.widgets.text("catalog", "retail_consumer_goods")
dbutils.widgets.text("schema", "conversational_commerce_search")
dbutils.widgets.text("model_serving_endpoint", "databricks-gte-large-en")
dbutils.widgets.text("vector_search_endpoint", "dbdemos_vs_endpoint")

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
model_serving_endpoint = dbutils.widgets.get("model_serving_endpoint")
vector_search_endpoint = dbutils.widgets.get("vector_search_endpoint")
print(f"{catalog}, {schema}, {model_serving_endpoint}, {vector_search_endpoint}")

In [0]:
import mlflow.deployments
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import numpy as np



## 1. Load Base Data & Create Embedding Text


In [0]:
prod_table = f"{catalog}.{schema}.products"
cust_table = f"{catalog}.{schema}.customers"
interactions_table = f"{catalog}.{schema}.interactions"
products_df = spark.table(prod_table)
customers_df = spark.table(cust_table)
interactions_df = spark.table(interactions_table)

# Create a rich text representation for product embeddings including image context
@udf(returnType=StringType())
def create_product_text(name, brand, category, subcategory, description, keywords, features):
    # Include AI image context in embedding text for better semantic representation
    # if image_source is None:
    #     image_context = "No image available"
    # else:
    # image_context = f"AI generated image available at {image_source}"
    return f"Product: {name} | Brand: {brand} | Category: {category}, {subcategory} | Description: {description} | Keywords: {keywords} | Features: {features}"

products_with_text_df = products_df.withColumn(
    "embedding_text",
    create_product_text(col("name"), col("brand"), col("category"), col("subcategory"), col("description"), col("keywords"), col("features"))
)
print("Created text for product embeddings with image context:")
display(products_with_text_df.select("product_id", "embedding_text"))


## 2. Generate Product & User Embeddings


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType

# 1. Collect all needed fields together to preserve alignment
product_rows = products_with_text_df.select(
    "product_id", 
    "embedding_text", 
    "brand", 
    "price",
    "category",
    "subcategory",
    "avg_rating"
).collect()

# 2. Extract texts for embeddings
product_texts = [row["embedding_text"] for row in product_rows]

# 3. Generate embeddings in batches
client = mlflow.deployments.get_deploy_client("databricks")
batch_size = 150
product_embeddings_list = []

for i in range(0, len(product_texts), batch_size):
    batch = product_texts[i:i + batch_size]
    response = client.predict(endpoint=model_serving_endpoint, inputs={"input": batch})
    product_embeddings_list.extend([item["embedding"] for item in response.data])

# Ensure embeddings are float arrays
product_embeddings_list = [[float(e) for e in emb] for emb in product_embeddings_list]

# 4. Build product_features_data by zipping rows with embeddings
product_features_data = []
for row, embedding in zip(product_rows, product_embeddings_list):
    product_features_data.append((
        row["product_id"],
        row["brand"],        # Brand filter
        row["price"],        # Price filter
        row["category"],     # Category filter
        row["subcategory"],  # Subcategory filter
        row["avg_rating"],   # Rating filter
        embedding
    ))

# 5. Define schema
product_features_schema = StructType([
    StructField("product_id", StringType()),
    StructField("brand", StringType()),
    StructField("price", DoubleType()),
    StructField("category", StringType()),      # Added category
    StructField("subcategory", StringType()),   # Added subcategory
    StructField("avg_rating", DoubleType()),    # Added avg_rating
    StructField("embedding", ArrayType(FloatType()))
])

# 6. Create DataFrame and save
product_features_df = spark.createDataFrame(product_features_data, schema=product_features_schema)

# Overwrite with new schema
product_features_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(
    f"{catalog}.{schema}.product_features"
)

print("✅ Product embeddings generated and saved to 'product_features' table with extended metadata.")

In [0]:
# Create user embeddings (weighted average of interacted product embeddings)
product_embeddings_map = {row['product_id']: row['embedding'] for row in product_features_df.collect()}
interactions_pd = interactions_df.toPandas()
user_embeddings = {}
for customer_id, group in interactions_pd.groupby('customer_id'):
    weighted_embeddings = []
    for _, row in group.iterrows():
        embedding = product_embeddings_map.get(row['product_id'])
        if embedding:
            # Simple average for this example
            weighted_embeddings.append(np.array(embedding))
    if weighted_embeddings:
        user_embeddings[customer_id] = np.mean(weighted_embeddings, axis=0).tolist()

user_features_df = spark.createDataFrame(
    [(k, v) for k, v in user_embeddings.items()],
    ["customer_id", "embedding"]
)
user_features_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalog}.{schema}.user_features")
print("✅ User embeddings generated and saved to 'user_features' table.")


In [0]:
spark.sql(f"ALTER TABLE {catalog}.{schema}.product_features SET TBLPROPERTIES (delta.enableChangeDataFeed = true);");
spark.sql(f"ALTER TABLE {catalog}.{schema}.user_features SET TBLPROPERTIES (delta.enableChangeDataFeed = true);");

## 3. Setup UC Vector Search


In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

endpoint_name = vector_search_endpoint
if endpoint_name not in [e['name'] for e in vsc.list_endpoints().get('endpoints', [])]:
    vsc.create_endpoint(name=endpoint_name, endpoint_type="STANDARD")
    print(f"Created Vector Search endpoint: {endpoint_name}")
else:
    print(f"Vector Search endpoint '{endpoint_name}' already exists.")

# Create Vector Search Index for Products
product_index_name = f"{catalog}.{schema}.product_index"
# MODIFICATION: Explicitly drop the index to ensure it rebuilds with new schema
try:
    vsc.delete_index(endpoint_name, product_index_name)
    print(f"Deleted existing index '{product_index_name}' to trigger rebuild.")
except Exception:
    pass # Index didn't exist
try:
    vsc.get_index(endpoint_name, product_index_name).describe()
    print(f"Product index '{product_index_name}' already exists.")
except Exception:
    print(f"Creating product index '{product_index_name}'...")
    vsc.create_delta_sync_index(
        endpoint_name=endpoint_name,
        index_name=product_index_name,
        source_table_name=f"{catalog}.{schema}.product_features",
        pipeline_type="TRIGGERED",
        primary_key="product_id",
        embedding_dimension=1024, 
        embedding_vector_column="embedding"
    )
    
# Create Vector Search Index for Users
user_index_name = f"{catalog}.{schema}.user_index"
try:
    vsc.get_index(endpoint_name, user_index_name).describe()
    print(f"User index '{user_index_name}' already exists.")
except Exception:
    print(f"Creating user index '{user_index_name}'...")
    vsc.create_delta_sync_index(
        endpoint_name=endpoint_name,
        index_name=user_index_name,
        source_table_name=f"{catalog}.{schema}.user_features",
        pipeline_type="TRIGGERED",
        primary_key="customer_id",
        embedding_dimension=1024, # Dimension for GTE-Large
        embedding_vector_column="embedding"
    )

print("\n✅ Vector Search setup is complete. You can now use the indexes for real-time lookups.")


In [0]:
spark.sql(f"ALTER TABLE {catalog}.{schema}.products SET TBLPROPERTIES (delta.enableChangeDataFeed = true);");

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

index = vsc.create_delta_sync_index(
  endpoint_name=f"{vector_search_endpoint}",
  source_table_name=f"{catalog}.{schema}.products",
  index_name=f"{catalog}.{schema}.product_desc_index",
  pipeline_type="TRIGGERED",
  primary_key="product_id",
  embedding_source_column="description",
  embedding_model_endpoint_name=model_serving_endpoint
)